Import necessary libraries

In [ ]:
library(fastscore)

# Model Definition

Define the input and output schemata, as well as the model.

In [ ]:
# Create an attachment
model_params <- list(a=1, b=2)
save(model_params, file='model_params.RData')

In [ ]:
# Input and output schemata are JSON strings
schin <- '{"type":"record", "name":"schin", "fields":[{"name":"x", "type":"int"}, {"name":"y", "type":"int"}]}'
schout <- '{"type":"record", "name":"schin", "fields":[{"name":"x", "type":"int"}, {"name":"y", "type":"int"}, {"name":"z", "type":"int"}]}'

# Model is a string of R code
modelstr <- '
# fastscore.input: schin
# fastscore.output: schout
# fastscore.recordsets: none

begin <- function(){
  load("model_params.RData")
  model_params <<- model_params # copy to global environment
}

action <- function(datum){
  # This is equally valid for records or data frames
  myd <- datum
  myd[["z"]] <- model_params[["a"]]*myd[["x"]] + model_params[["b"]]*myd[["y"]]
  emit(myd)
}
'

Create the model object, passing the global environment to bind `begin()` and `action()`

In [ ]:
mymodel <- Model_from_string(modelstr, environment())

Score some sample inputs using JSON:

In [ ]:
mymodel$score(list('{"x":2, "y":3}', '{"x":4, "y":8}'), use_json=TRUE)

Validate against expected outputs (note the schema error in the second output):

In [ ]:
mymodel$validate(list('{"x":2, "y":3}', '{"x":4, "y":8}'), list('{"x":2,"y":3,"z":5}', '{"x":4,"y":8,"z":12}'), use_json=TRUE)

Record sets are also supported.

In [ ]:
# change the option
mymodel$options[['recordsets']] <- 'both'

# create a dataframe
mydf <- data.frame(x=c(1, 2, 3, 4), y=c(6, 7, 8, 9))
mymodel$score(mydf)

We can also use JSON inputs.

In [ ]:
# convert the dataframe to JSON
inputs <- recordset_to_json(mydf, mymodel$input_schema)

mymodel$score(inputs, use_json=TRUE)

Rebind the model's begin() or action methods:

In [ ]:
mymodel$begin <- function(){
    load("model_params.RData")
    model_params <<- model_params # copy to global environment
    message("Now we're cooking with fire")
}
mymodel$score(mydf)

Let's take a look at what the model will look like in FastScore.

In [ ]:
mymodel$to_string()

Don't forget to include the attachment with the model data! Attachment bundles must be either .zip or .tar.gz files.

In [ ]:
system('tar czvf rattachment.tar.gz model_params.RData')
mymodel$attachments[['model_params']] <- 'rattachment.tar.gz'

# FastScore Engines

The `Engine` class lets you interact directly with a running engine.

In [ ]:
engine <- Engine(proxy_prefix='https://localhost:8000', container='engine-x-3')

`Model` objects can be deployed directly to an `Engine` (check the FastScore Dashboard)

In [ ]:
mymodel$name <- 'example_R_model'
engine$deploy(mymodel)

Automatic JSON serialization lets you pass objects directly to a running `Engine` for scoring.

In [ ]:
engine$score(mydf)

Or, you can use JSON-encoded inputs

In [ ]:
engine$score(inputs, use_json=TRUE)

When you're done, stop the engine and unload the model.

In [ ]:
engine$stop()